In [92]:
from scipy.io import loadmat
import os
import json
import numpy as np

In [93]:
def load(file, split="train", truncate = 9000):
    d = loadmat(file, squeeze_me=True, struct_as_record=True)
    annot = d["hgS_070000"]["properties"][()]["ApplicationData"][()]["DatatipInformation"][()]
    ys = d["hgS_070000"]["children"][()]["children"][0]["properties"][0]["YData"][()][:truncate].tolist()
    name = file.split(".")[0].split("/")[1] 
    return { "signal": ys, "csp_label": sorted([annot[0]["Position"][0], annot[1]["Position"][0]]), "id": name , "split": split }

def save(files, json_file, labeler, **kwargs):
    res = []
    for file in files:
        if file.endswith(".fig"):
            name = file.split("/")[1].split(".")[0]
            split = labeler(name)
            res.append(load(file, split=split, **kwargs))
    json.dump(res, open(json_file, "w"))

In [95]:
known_labels = {e["id"]: e["split"] for e in json.load(open('test_train_split.json'))}
files = [f"MatlabFigures/{file}" for file in os.listdir("MatlabFigures")]
filesKatrin = [file for file in files if "Katrin" in file]
filesAndre = [file for file in files if "Andre" in file]
def labeler(name):
    return known_labels[name]
save(files, "all_data.json", labeler, truncate=3000)
save(filesKatrin, "katrin_data.json", labeler, truncate=3000)
save(filesAndre, "andre_data.json", labeler, truncate=3000)